In [ ]:
# wsi处理 
step1 = False
step2 = False
step3 = True
## step1 清理低质量图
import openslide 
import os 
import shutil
import pandas as pd

path = r'/home/sci/Disk2/tcga_crc/DATA_DIRECTORY'
trash_path = r'/home/sci/Disk2/tcga_crc/trash'
slides = os.listdir(path)
slides = [slide for slide in slides if os.path.isfile(os.path.join(path,slide))]
# print(slides)
i = 0
if step1:
    for slide in slides:
        slide_path = os.path.join(path,slide)
        slide_object = openslide.OpenSlide(slide_path)
        if slide_object.level_count == 2:
            shutil.move(slide_path,trash_path)
            i += 1
            print(f'already process {i} slides to trash')

## step 2 20/40 划分
if step2:
    for slide in slides:
        slide_path = os.path.join(path,slide)
        slide_object = openslide.OpenSlide(slide_path)
        if slide_object.properties['aperio.AppMag'] == '40':
            shutil.move(slide_path, os.path.join(path,'40'))
        else:
            shutil.move(slide_path, os.path.join(path,'20'))
        i += 1
        print(f'already process {i} slides to magfilefolder')
## 删除不能用的WSI
if step3:
    
    data_dir = r'/home/sci/Disk_data/Datasets/TCGA-NSCLC/WSI'
    # trash_20 = r'/home/sci/Disk2/tcga_crc/trash/trash_20'
    # os.makedirs(trash_20,exist_ok=True)
    all_slides = os.listdir(data_dir)
    patch_dir = r'/home/sci/Disk_data/Datasets/TCGA-NSCLC/BLOCKS_level-1/patches'
    feat_dir = r'/home/sci/Disk_data/Datasets/TCGA-NSCLC/FEATURES_levle-1/pt_files'
    all_patches = os.listdir(patch_dir)
    for slide in all_slides:
        slide_name,_ = os.path.splitext(slide)
        slide_path = os.path.join(data_dir,slide)
        # patch_path = os.path.join(patch_5_dir,f'{slide_name}.h5')
        if not os.path.exists(os.path.join(patch_dir,f'{slide_name}.h5')):
            shutil.move(slide_path, '/home/sci/Disk_data/Datasets/error')
            os.remove(f'/home/sci/Disk_data/Datasets/TCGA-NSCLC/BLOCKS_level-1/masks/{slide_name}.jpg')
            # os.remove(f'/home/sci/Disk_data/Datasets/TCGA-NSCLC/BLOCKS/thumbnails/{slide_name}_thumb.jpg')


In [ ]:
## csv_file process
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import os

df_csv = pd.read_csv('dataset_csv/TCGA-NSCLC.csv')
dataset_dir = '/home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/WSI'
index = df_csv.index
rows_to_delete = []
for name in os.listdir(dataset_dir):
    slide_id = os.path.splitext(name)[0]

    row_index = df_csv[df_csv['slide_id'] == slide_id].index
    rows_to_delete.append(row_index.item())
    
remaining_index = index.difference(rows_to_delete)
df_delete = df_csv.drop(remaining_index).reset_index(drop=True)
df_delete
# df = df_csv.drop(~rows_to_delete)

df_delete.to_csv('dataset_csv/tcga_lung_subtyping_sub.csv')

In [ ]:
# WSI 属性查看
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import openslide
import os
import torch
slide_path = r'/home/sci/Disk_data/Datasets/TCGA-NSCLC/WSI/TCGA-05-4244-01A-01-BS1.svs'
path = r'/home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/WSI'
# # ---->> for single slide
# slide = openslide.OpenSlide(slide_path)
# slide.dimensions
# slide.associated_images
# slide.level_dimensions
# slide.level_count
# slide.properties['aperio.AppMag']
# slide.properties['aperio.MPP']

# ---->> for slide folder
slides = os.listdir(path)
dimentions_list = []
for slide in slides:
    slide = os.path.join(path,slide)
    slide = openslide.OpenSlide(slide)
    print(slide.level_dimensions)
    print(slide.properties['aperio.AppMag'])
    print(slide.level_count)
    # slide = slide.get_thumbnail((slide.level_dimensions[-1]))
    # slide.save(r'/home/sci/Disk2/tcga_crc/DATA_DIRECTORY/test')
    # print('save_success')
    # dimentions_list.append(slide.level_dimensions[-1])
# dimentions_list=torch.tensor(dimentions_list)
# index = torch.topk(dimentions_list, 1, dim=1)
# maxMap = torch.index_select(dimentions_list, index=index, dim=0 )
# maxMap




In [ ]:
# ----> wsi transfer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import openslide
import os
import torch
import glob
import shutil

path = r"/home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/WSI"
path_20 = os.path.join(path, '20')
path_40 = os.path.join(path, '40')
os.makedirs(path_20, exist_ok=True)
os.makedirs(path_40, exist_ok=True)
slide_list = glob.glob(os.path.join(path, '*.svs'))

# for slide_path in slide_list:
#     # slide_path = os.path.join(path, slide)
#     slide_obj = openslide.open_slide(slide_path)

#     if slide_obj.level_count == 3:
#         shutil.move(slide_path, path_20)
#     elif slide_obj.level_count == 4:
#         shutil.move(slide_path, path_40)

# ----> trans back
allslide_path = glob.glob(os.path.join(path, '*/*.svs'))
for slide_path in allslide_path:
    shutil.move(slide_path, path)



In [ ]:
import h5py
import os
path = r'/home/sci/Disk2/CPTAC-LUNG/FEATURES_level3/h5_files'
num_patches = 0
num_wsis = 0
for i, h5_file in enumerate(os.listdir(path)):
    fpath = os.path.join(path, h5_file)
    f = h5py.File(fpath,'r')
    # f.keys()
    data = f.get('coords')
    # print(data)
    num_patches += data.shape[0]
    num_wsis += 1
print(f'Patches: {num_patches}')
print(f'Average patches: {num_patches/num_wsis}')


## Step_1 get patch

source svs 文件地址\
save_dir 结果文件\

In [ ]:
'''
python create_patches_fp.py \
--source 'svs_dir' \
--save_dir 'result_dir' \
--patch_size 256 \
--seg \ # 分割=True
--patch \ # 切分patch=True
--stitch \ # 
process_list_autogen.csv # 这里记录了每张 image 的处理的参数
'''


In [3]:
##改 savedir and patch_level
!python create_patches_fp.py --source /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/WSI/40 --save_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2 --patch_level 2 --patch_size 256 --seg --patch --stitch

source:  /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/WSI/40
patch_save_dir:  /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2/patches
mask_save_dir:  /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2/masks
stitch_save_dir:  /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2/stitches
thumbnail_save_dir:  /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2/thumbnails
source : /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/WSI/40
save_dir : /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2
patch_save_dir : /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2/patches
mask_save_dir : /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2/masks
stitch_save_dir : /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2/stitches
thumbnail_save_dir : /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2/thumbnails
{'seg_params': {'seg_level': -1, 'sthresh': 8, 'mthresh': 7, 'close': 4, 'use_otsu': False, 'keep_ids': 'none', 'exclude_ids': 'none'},

## Step_2 get patch features
data_h5_dir  输出文件地址\
data_slide_dir svs文件地址\
上一步生成的csv csv_path\
feat_dir 输出文件地址\


In [ ]:
'''
CUDA_VISIBLE_DEVICES=0 python extract_features_fp.py \
--data_h5_dir data/RESULTS_DIRECTORY/patches \
--data_slide_dir /media/yuansh/14THHD/CLAM/DataSet/toy_example \
--csv_path /media/yuansh/14THHD/CLAM/Step_2.csv \
--feat_dir /media/yuansh/14THHD/CLAM/FEATURES_DIRECTORY \
--batch_size 512 \
--slide_ext .svs
'''
!python extract_features_fp.py\
    --data_h5_dir E:\Workspace\Project\CLAM\data\RESULTS_DIRECTORY \
    --data_slide_dir F:/Download/TCGA \
    --csv_path data\RESULTS_DIRECTORY\Step_2.csv \
    --feat_dir E:\Workspace\Project\CLAM\data\FEATURES_DIRECTORY \
    --batch_size 256 \
    --slide_ext .svs


In [ ]:
# 生成第2步骤需要的csv文件
from utils.csv_gen import *

csv_dir = r'/home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level1/process_list_autogen.csv'
# sort_csv = pd.read_csv(csv_dir).sort_values('slide_id')
result_dir = r'/home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level1/step2_get_features.csv'
patch_dir = r'/home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level1/patches'
csv_gen_step1(csv_dir,result_dir,patch_dir)

In [5]:
!CUDA_VISIBLE_DEVICES=0 python extract_features_fp.py --data_h5_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2 \
--data_slide_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/WSI \
--csv_path /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2/step2_get_features.csv \
--feat_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/FEATURES_level2 --batch_size 512 --slide_ext .svs

initializing dataset
loading model checkpoint

progress: 0/828
TCGA-18-3406-01Z-00-DX1
downsample [16.00012473 16.00091158]
downsampled_level_dim [8017 5485]
level_dim [8017 5485]
name TCGA-18-3406-01Z-00-DX1
patch_level 2
patch_size 256
save_path /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2/patches

feature extraction settings
target patch size:  None
pretrained:  True
transformations:  Compose(
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)
processing /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/BLOCKS_level2/patches/TCGA-18-3406-01Z-00-DX1.h5: total of 1 batches
batch 0/1, 0 files processed

computing features for /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/FEATURES_level2/h5_files/TCGA-18-3406-01Z-00-DX1.h5 took 6.622757196426392 s
features size:  (423, 1024)
coordinates size:  (423, 2)

progress: 1/828
TCGA-18-3407-01Z-00-DX1
downsample [16.00013701 16.        ]
downsampled_level_dim [7299 5005]
level_dim [7299 5005]
name TCGA-18-

## step3 Create split
注意修改Create_split_seq.py文件中的csv路径

In [ ]:
from utils.csv_gen import * 
path = r'/home/sci/Disk_data/TCGA-NSCLC/WSI'
# sort_csv = pd.read_csv(csv_dir).sort_values('slide_id')
result_dir = r'/home/sci/Disk_data/TCGA-NSCLC/RESULTS_DIRECTORY/step3_get_splits.csv' ## 5 + 20
patch_dir = r'/home/sci/Disk_data/TCGA-NSCLC/RESULTS_DIRECTORY/patches'
csv_gen_test(path,result_dir)

In [4]:
!python create_splits_seq.py --task task_2_tumor_subtyping --seed 1 --label_frac 1 --k 5 --csv_path dataset_csv/tcga_lung_subtyping_sub.csv

/home/sci/PycharmProjects/chaofan/projects/CLAM/datasets/dataset_generic.py:104: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  label = stats.mode(label)[0]
/home/sci/PycharmProjects/chaofan/projects/CLAM/datasets/dataset_generic.py:104: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avo

## Step4 Train
注意修改main.py中的csv路径

CUDA_VISIBLE_DEVICES=0 python main.py \
--drop_out \
--early_stopping \
--lr 2e-4 \
--k 10 \
--label_frac 0.75 \
--exp_code task_1_tumor_vs_normal_CLAM_50 --weighted_sample --bag_loss ce --inst_loss svm --task task_1_tumor_vs_normal --model_type clam_sb --log_data \
--data_root_dir /media/yuansh/14THHD/CLAM/FEATURES_DIRECTORY



In [ ]:
!CUDA_VISIBLE_DEVICES=0 python main.py --drop_out --early_stopping --lr 2e-4 --k 5 --label_frac 1\
--exp_code tcga_nsclc_sub_100_level3_mb --weighted_sample --bag_loss ce --inst_loss svm \
--task task_2_tumor_subtyping --model_type clam_mb --log_data --data_root_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/FEATURES_level3 \
--split_dir /home/sci/Disk_data/SCI_Projects/chaofan/projects/CLAM/splits/tcga_nsclc_100 --subtyping \
--csv_path dataset_csv/tcga_lung_subtyping_sub.csv

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python main.py --drop_out --early_stopping --lr 2e-4 --k 5 --label_frac 0.75 \
--exp_code tcga_nsclc_75_level1_transmil --weighted_sample --bag_loss ce --inst_loss svm \
--task task_2_tumor_subtyping --model_type transmil --log_data --data_root_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC/FEATURES_level1 \
--split_dir /home/sci/Disk_data/SCI_Projects/chaofan/projects/CLAM/splits/task_2_tcga_subtyping_75 --subtyping

### TransformerMIL实验结果

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python main.py --drop_out --early_stopping --lr 2e-4 --k 5 --label_frac 1 \
--exp_code tcga_nsclc_sub_100_level3_transmil --weighted_sample --bag_loss ce --inst_loss svm \
--task task_2_tumor_subtyping --model_type transmil --log_data --data_root_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/FEATURES_level3 \
--split_dir /home/sci/PycharmProjects/chaofan/projects/CLAM/splits/tcga_nsclc_100 --subtyping \
--csv_path dataset_csv/tcga_lung_subtyping_sub.csv

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python main.py --drop_out --early_stopping --lr 2e-4 --k 5 --label_frac 1 \
--exp_code tcga_nsclc_sub_100_level1_transmil --weighted_sample --bag_loss ce --inst_loss svm \
--task task_2_tumor_subtyping --model_type transmil --log_data --data_root_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/FEATURES_level1 \
--split_dir /home/sci/Disk_data/SCI_Projects/chaofan/projects/CLAM/splits/tcga_nsclc_100 --subtyping \
--csv_path dataset_csv/tcga_lung_subtyping_sub.csv

### proposed

In [1]:
!CUDA_VISIBLE_DEVICES=0 python main_myself.py --drop_out --early_stopping --lr 2e-4 --max_lr 2e-3 --k 5 --label_frac 1 --opt adam\
--exp_code tcga_nsclc_sub_100_level13_MCBAT_d1f2l3 --weighted_sample --bag_loss ce --inst_loss svm \
--task task_2_tumor_subtyping --model_type mcbat_sb --log_data --data_low_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/FEATURES_level3 \
--data_high_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/FEATURES_level1 \
--split_dir /home/sci/PycharmProjects/chaofan/projects/CLAM/splits/tcga_nsclc_100 --subtyping \
--csv_path dataset_csv/tcga_lung_subtyping_sub.csv 


Load Dataset
label column: label
label dictionary: {'LUAD': 0, 'LUSC': 1}
number of classes: 2
slide-level counts:  
 0    394
1    434
Name: label, dtype: int64
Patient-LVL; Number of samples registered in class 0: 333
Slide-LVL; Number of samples registered in class 0: 394
Patient-LVL; Number of samples registered in class 1: 400
Slide-LVL; Number of samples registered in class 1: 434
split_dir:  /home/sci/PycharmProjects/chaofan/projects/CLAM/splits/tcga_nsclc_100
################# Settings ###################
num_splits:  5
k_start:  -1
k_end:  -1
task:  task_2_tumor_subtyping
max_epochs:  200
results_dir:  ./results
lr:  0.0002
experiment:  tcga_nsclc_sub_100_level13_MCBAT_d1f2l3
reg:  1e-05
label_frac:  1.0
bag_loss:  ce
seed:  1
model_type:  mcbat_sb
model_size:  small
use_drop_out:  True
weighted_sample:  True
opt:  adam
bag_weight:  0.7
inst_loss:  svm
B:  8
split_dir:  /home/sci/PycharmProjects/chaofan/projects/CLAM/splits/tcga_nsclc_100

Training Fold 0!

Init train/val/tes

## eval

CUDA_VISIBLE_DEVICES=0 python eval.py \
--drop_out \
--k 10 \
--models_exp_code task_1_tumor_vs_normal_CLAM_50_s1 \
--save_exp_code task_1_tumor_vs_normal_CLAM_50_s1_cv \
--task task_1_tumor_vs_normal \
--model_type clam_sb \
--results_dir results \
--data_root_dir /media/yuansh/14THHD/CLAM/FEATURES_DIRECTORY


In [15]:
!python eval.py --drop_out --k 5 --models_exp_code tcga_nsclc_sub_100_level1_mil_s1 \
    --save_exp_code tcga_nsclc_100_mil_fornlst100 \
    --task task_2_tumor_subtyping --model_type mil --results_dir results \
    --data_root_dir /home/sci/Disk_data/Datasets/NLST/FEATURES_level1\
    --csv_path dataset_csv/NLST_offical.csv\
    --splits_dir /home/sci/PycharmProjects/chaofan/projects/CLAM/splits/NLST_100/

{'task': 'task_2_tumor_subtyping', 'split': 'test', 'save_dir': './eval_results/EVAL_tcga_nsclc_100_mil_fornlst100', 'models_dir': 'results/tcga_nsclc_sub_100_level1_mil_s1', 'model_type': 'mil', 'drop_out': True, 'model_size': 'small'}
label column: label
label dictionary: {'LUAD': 0, 'LUSC': 1}
number of classes: 2
slide-level counts:  
 0    552
1    324
Name: label, dtype: int64
Patient-LVL; Number of samples registered in class 0: 194
Slide-LVL; Number of samples registered in class 0: 552
Patient-LVL; Number of samples registered in class 1: 117
Slide-LVL; Number of samples registered in class 1: 324
Init Model
MIL_fc(
  (classifier): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=512, out_features=2, bias=True)
  )
)
Total number of parameters: 525826
Total number of trainable parameters: 525826
Init Loaders
Traceback (most recent call last):
  File "/home/sci/PycharmP

In [2]:
!python eval_myself.py --drop_out --k 5 --models_exp_code tcga_nsclc_sub_100_level13_mcbat_sb_depth1_s1 \
    --save_exp_code tcga_nsclc_100_mcbat_sb \
    --task task_2_tumor_subtyping --model_type mcbat_sb --results_dir results \
    --data_low_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/FEATURES_level3\
    --data_high_dir /home/sci/Disk_data/Datasets/TCGA-NSCLC-sub/FEATURES_level1\
    --csv_path dataset_csv/tcga_lung_subtyping_sub.csv\
    --splits_dir /home/sci/PycharmProjects/chaofan/projects/CLAM/splits/tcga_nsclc_100/

{'task': 'task_2_tumor_subtyping', 'split': 'test', 'save_dir': './eval_results/EVAL_tcga_nsclc_100_mcbat_sb', 'models_dir': 'results/tcga_nsclc_sub_100_level13_mcbat_sb_depth1_s1', 'model_type': 'mcbat_sb', 'drop_out': True, 'model_size': 'small'}
label column: label
label dictionary: {'LUAD': 0, 'LUSC': 1}
number of classes: 2
slide-level counts:  
 0    394
1    434
Name: label, dtype: int64
Patient-LVL; Number of samples registered in class 0: 333
Slide-LVL; Number of samples registered in class 0: 394
Patient-LVL; Number of samples registered in class 1: 400
Slide-LVL; Number of samples registered in class 1: 434
Init Model
MCBAT_SB(
  (instance_loss_fn): CrossEntropyLoss()
  (attention_net): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Attn_Net_Gated(
      (attention_a): Sequential(
        (0): Linear(in_features=512, out_features=256, bias=True)
        (1): Tanh()
        (2): Dropo

In [ ]:
!python create_heatmaps.py --config config_template.yaml